In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_1samp, shapiro
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import recall_score, make_scorer, roc_auc_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from graphviz import Source
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG, display
# import shap

import warnings
warnings.filterwarnings('ignore')
from scipy.stats.mstats import winsorize
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Applying PCA on Original

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ml1_final_project/data_cardiovascular_risk.csv')
columns_to_convert = ['sex', 'education', 'is_smoking', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'TenYearCHD']
df[columns_to_convert] = df[columns_to_convert].astype('object')
categorical_columns = df.select_dtypes(include='object').columns
numerical_columns = df.select_dtypes(include=['float', 'int']).columns

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Remove rows where 'bpmeds' column is null
df = df.dropna(subset=['BPMeds'])

imputing_median1 = df[df['is_smoking'] == 'YES']['cigsPerDay'].median()
df['cigsPerDay'] = df['cigsPerDay'].fillna(imputing_median1)


# Dropping the 3 Columns #######################################################
df.drop('id', axis=1, inplace=True)
df.drop('education', axis=1, inplace=True)
df.drop(['is_smoking', 'prevalentStroke'], axis=1, inplace=True)
##################################################################################

for var in ['totChol', 'BMI', 'heartRate']:
  imputer = IterativeImputer(random_state=0)
  df[[var]] = imputer.fit_transform(df[[var]])


# fit and transform the imputer on your data
imputer = IterativeImputer(random_state = 0)
imputer.fit(df[['glucose']])

df[['glucose']] = imputer.transform(df[['glucose']])


##### New Feature meanBP
# Create a new feature 'mean_BP' as the mean of 'sysBP' and 'diaBP'
df['mean_BP'] = (df['sysBP'] + df['diaBP']) / 2

# Drop the 'sysBP' and 'diaBP' columns
df.drop(['sysBP', 'diaBP'], axis=1, inplace=True)
###########

df = pd.get_dummies(df, drop_first=True)

X = df.drop(['TenYearCHD_1'], axis = 1)
y = df['TenYearCHD_1']

X_winsorized = X.copy()


# Apply winsorization to each column with the optimal limits
X_winsorized = X.copy()

X_winsorized['age'] = winsorize(X_winsorized['age'], limits=(0, 0))
X_winsorized['cigsPerDay'] = winsorize(X_winsorized['cigsPerDay'], limits=(0.7, 0.01))
X_winsorized['totChol'] = winsorize(X_winsorized['totChol'], limits=(0, 0))
# X_winsorized['sysBP'] = winsorize(X_winsorized['sysBP'], limits=(0, 0))
# X_winsorized['diaBP'] = winsorize(X_winsorized['diaBP'], limits=(0.8, 0))
X_winsorized['mean_BP'] = winsorize(X_winsorized['mean_BP'], limits=(0, 0))
X_winsorized['BMI'] = winsorize(X_winsorized['BMI'], limits=(0, 0))
X_winsorized['heartRate'] = winsorize(X_winsorized['heartRate'], limits=(0, 0))
X_winsorized['glucose'] = winsorize(X_winsorized['glucose'], limits=(0.7, 0.2))
X_winsorized['sex_M'] = winsorize(X_winsorized['sex_M'], limits=(0, 0))
X_winsorized['BPMeds_1.0'] = winsorize(X_winsorized['BPMeds_1.0'], limits=(0, 0))
X_winsorized['prevalentHyp_1'] = winsorize(X_winsorized['prevalentHyp_1'], limits=(0, 0.4))
X_winsorized['diabetes_1'] = winsorize(X_winsorized['diabetes_1'], limits=(0, 0))


#################################################################################
# Create an instance of PCA
pca = PCA(n_components=8)  # Set the number of components (dimensions) you want to keep

# Fit the PCA model to your data
pca.fit(X_winsorized)

# Transform the data to the lower-dimensional space
X_pca = pca.transform(X_winsorized)
###################################################################################

# Split the winsorized data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# Train Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train_s, y_train)

# Make predictions and calculate AUC
y_pred = gnb.predict_proba(X_test_s)[:, 1]
auc = roc_auc_score(y_test, y_pred)

# Print the AUC score
print("AUC:", auc)


AUC: 0.737537757437071


In [ ]:
X_winsorized.shape

(3346, 16)